# Wage Prediction using Lasso

This notebook is based on a lab from Chapter 1 of the book [Applied Causal Inference Powered by ML and AI](https://causalml-book.org). We will answer what determines the wage of workers from a predictive perspective.

This example focuses on a sample of Registered Nurses in the US collected during 2017. The hourly wage of a nurse is denoted by $Y$ and $X$ is a vector of nurses' characteristics, e.g., human capital, demographics, job-relevant characteristics. The question that we want to answer is:

- How to use nurses' characteristics, such as education and experience, to best predict wages?


In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin

import warnings
warnings.simplefilter('ignore')

## Data

In [2]:
# Read data
path_data = "https://github.com/pabloestradac/causalml-basics/raw/main/data/"
df = pd.read_csv(path_data + 'wages_nurses.csv')
df.tail()

,lwage,female,age,race,children,marital,education,RN_experience,left_nursing,country_ed_US,english_only,military,certificates,labor_union,work_setting,work_situation,level_care,care_specialty,state
5617,3.496087,1,72,white,0,widowed_divorced_sep,ed_assoc,51,left_0,1,1,never_served_mil,0,0,SET_long_term_inpatient,SIT_self_employed,LC_nursing_home,CC_no_patient_care,state_n13
5618,3.678408,1,50,white,0,never_married,ed_msn,14,left_0,1,1,never_served_mil,0,0,SET_hospital,SIT_agency_facility,LC_education,CC_emergency_care,state_n24
5619,3.952845,0,36,white,0,currently_married,ed_bsn,4,left_0,1,1,never_served_mil,2,1,SET_hospital,SIT_agency_facility,LC_inpatient,CC_oncology,state_n32
5620,3.957960,1,61,white,0,currently_married,ed_bsn,40,left_0,1,1,never_served_mil,0,0,SET_hospital,SIT_agency_facility,LC_inpatient,CC_no_patient_care,state_n37
5621,3.100270,1,38,white,0,currently_married,ed_assoc,3,left_0,1,1,never_served_mil,2,0,SET_hospital,SIT_agency_facility,LC_others,CC_medical_surgical,state_n29


Construct a prediction rule for hourly (log) wage $Y$, which depends linearly on relevant characteristics $X$:

$$
\log(\text{wage}) = \beta' X + e
$$

Then, assess the predictive performance of a given model using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$. Thus, we measure the prediction quality of the models via data splitting.


## Basic vs Flexible Model

We employ two different specifications for prediction:

1. Basic Model: $X$ consists of a set of raw regressors

2. Flexible Model: $X$ consists of all raw regressors from the basic model plus a dictionary of transformations.

Let's start with OLS.

In [3]:
# 0. Traditional Model
model_base = ('lwage ~ female + age + race + children + marital + education + RN_experience')
base = smf.ols(model_base, data=df)
results_base = base.fit()

In [4]:
X, y = base.data.exog, base.data.endog
rsquared_base = results_base.rsquared
rsquared_adj_base = results_base.rsquared_adj
mse_base = np.mean(results_base.resid**2)
mse_adj_base = results_base.mse_resid
print(f'No_Features = {X.shape[1]:.0f}')
print(f'Rsquared = {rsquared_base:.3f}')
print(f'Rsquared_adjusted = {rsquared_adj_base:.3f}')
print(f'MSE = {mse_base:.3f}')
print(f'MSE_adjusted = {mse_adj_base:.3f}')

No_Features = 14
Rsquared = 0.084
Rsquared_adjusted = 0.082
MSE = 0.116
MSE_adjusted = 0.116


In [5]:
# 1. Basic Model
model_base = ('lwage ~ female + age + race + children + marital '
              '+ education + RN_experience + left_nursing + country_ed_US + english_only + military + certificates'
              '+ labor_union + work_setting + work_situation + level_care + care_specialty + state')
base = smf.ols(model_base, data=df)
results_base = base.fit()

In [6]:
X, y = base.data.exog, base.data.endog
rsquared_base = results_base.rsquared
rsquared_adj_base = results_base.rsquared_adj
mse_base = np.mean(results_base.resid**2)
mse_adj_base = results_base.mse_resid
print(f'No_Features = {X.shape[1]:.0f}')
print(f'Rsquared = {rsquared_base:.3f}')
print(f'Rsquared_adjusted = {rsquared_adj_base:.3f}')
print(f'MSE = {mse_base:.3f}')
print(f'MSE_adjusted = {mse_adj_base:.3f}')

No_Features = 96
Rsquared = 0.289
Rsquared_adjusted = 0.276
MSE = 0.090
MSE_adjusted = 0.091


In [7]:
# 2. Flexible Model
model_flex = ('lwage ~ female + age + race + children + marital '
              '+ education + left_nursing + country_ed_US + english_only + military + certificates'
              '+ labor_union + work_setting + work_situation + level_care + care_specialty + state'
              '+ (RN_experience + RN_experience**2 + RN_experience**3 + RN_experience**4)'
              '*(age + race + children + marital '
              '+ education + left_nursing + country_ed_US + english_only + military + certificates'
              '+ labor_union + work_setting + work_situation + level_care + care_specialty + state)')
flex = smf.ols(model_flex, data=df)
results_flex = flex.fit()

In [8]:
X, y = flex.data.exog, flex.data.endog
rsquared_flex = results_flex.rsquared
rsquared_adj_flex = results_flex.rsquared_adj
mse_flex = np.mean(results_flex.resid**2)
mse_adj_flex = results_flex.mse_resid
print(f'No_Features = {X.shape[1]:.0f}')
print(f'Rsquared = {rsquared_flex:.3f}')
print(f'Rsquared_adjusted = {rsquared_adj_flex:.3f}')
print(f'MSE = {mse_flex:.3f}')
print(f'MSE_adjusted = {mse_adj_flex:.3f}')

No_Features = 189
Rsquared = 0.325
Rsquared_adjusted = 0.302
MSE = 0.085
MSE_adjusted = 0.088


## Flexible model using Lasso

Lasso (least absolute shrinkage and selection operator) minimizes the following objective function:

$$
\min_{\beta} \left\{ \sum_{i=1}^n (y_i - x_i^T \beta)^2 + \lambda \sum_{j=1}^p |\beta_j| \right\}
$$

where $\lambda$ is the regularization hyperparameter. We can use cross-validation to tune $\lambda$.

In practice, it is recommended to always standardize the regressors before fitting the Lasso model. This is because the penalty term is applied to the absolute value of the coefficients, and the scale of the regressors can affect the magnitude of the coefficients. To that properly, we will use a pipeline to standardize the regressors and fit the Lasso model.

See [Pipeline: chaining estimators](https://scikit-learn.org/stable/modules/compose.html#pipeline).

In [9]:
X = flex.data.exog[:, 1:] # exclude the intercept
y = flex.data.endog

# 3. Train model using Lasso with cross validation and variable normalization
lasso = Pipeline([('scale', StandardScaler()),
                  ('lasso', LassoCV())])
lasso.fit(X, y)

Pipeline(steps=[('scale', StandardScaler()), ('lasso', LassoCV())])

In [10]:
n, p = X.shape
p += 1
mse_lasso = np.mean((y - lasso.predict(X))**2)
mse_adj_lasso = mse_lasso * n / (n - p)
rsquared_lasso = 1 - mse_lasso / np.var(y)
rsquared_adj_lasso = 1 - mse_adj_lasso / np.var(y)
print(f'No_Features = {p:.0f}')
print(f'Rsquared = {rsquared_lasso:.3f}')
print(f'Rsquared_adjusted = {rsquared_adj_lasso:.3f}')
print(f'MSE = {mse_lasso:.3f}')
print(f'MSE_adjusted = {mse_adj_lasso:.3f}')

No_Features = 189
Rsquared = 0.316
Rsquared_adjusted = 0.292
MSE = 0.086
MSE_adjusted = 0.089


In [11]:
# Store results
res_df = pd.DataFrame()

res_df['Model'] = ['Basic reg', 'Flexible reg', 'Flexible Lasso']
res_df['p'] = [results_base.params.shape[0],
               results_flex.params.shape[0],
               results_flex.params.shape[0]]
res_df['R2'] = [rsquared_base, rsquared_flex, rsquared_lasso]
res_df['MSE'] = [mse_base, mse_flex, mse_lasso]
res_df['adj_R2'] = [rsquared_adj_base, rsquared_adj_flex, rsquared_adj_lasso]
res_df['adj_MSE'] = [mse_adj_base, mse_adj_flex, mse_adj_lasso]

res_df.round(3)

,Model,p,R2,MSE,adj_R2,adj_MSE
0,Basic reg,96,0.289,0.090,0.276,0.091
1,Flexible reg,189,0.325,0.085,0.302,0.088
2,Flexible Lasso,189,0.316,0.086,0.292,0.089


The flexible model performs slightly better than the basic model.

Let's now use sample splitting.

## Out-of-sample performance

Now that we have seen in-sample fit, we evaluate our models on the out-of-sample performance.


In [12]:
# Use smf.ols just to get the full data frame and sm.OLS to test out of sample for convenience
tmp = smf.ols(model_base, data=df)
X_full = tmp.data.exog
y_full = tmp.data.endog
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=.2, random_state=42)

# Predict out of sample
reg_basic = sm.OLS(y_train, X_train).fit()
yhat_reg_base = reg_basic.predict(X_test)

# Calculate out-of-sample MSE
MSE_test1 = sum((y_test-yhat_reg_base)**2)/y_test.shape[0]
R2_test1 = 1. - MSE_test1/np.var(y_test)

print(f'Test MSE for the basic model: {MSE_test1:.3f}')
print(f'Test R2 for the basic model: {R2_test1:.3f}')

Test MSE for the basic model: 0.101
Test R2 for the basic model: 0.256


In [13]:
# Use smf.ols just to get the full data frame and sm.OLS to test out of sample for convenience
tmp = smf.ols(model_flex, data=df)
X_full = tmp.data.exog
y_full = tmp.data.endog
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=.2, random_state=42)

# Predict out of sample
reg_flex = sm.OLS(y_train, X_train).fit()
yhat_reg_flex = reg_flex.predict(X_test)

# Calculate out-of-sample MSE
MSE_test2 = np.mean((y_test - yhat_reg_flex)**2)
R2_test2 = 1. - MSE_test2 / np.var(y_test)

print(f'Test MSE for the flexible model: {MSE_test2:.3f}')
print(f'Test R2 for the flexible model: {R2_test2:.3f}')

Test MSE for the flexible model: 0.099
Test R2 for the flexible model: 0.272


In [14]:
# Predict out of sample
lasso = Pipeline([('scale', StandardScaler()),
                  ('lasso', LassoCV())])
lasso.fit(X_train[:, 1:], y_train)
yhat_test_lasso = lasso.predict(X_test[:, 1:])

# Calculate out-of-sample MSE
MSE_test3 = np.mean((y_test - yhat_test_lasso)**2)
R2_test3 = 1. - MSE_test3 / np.var(y_test)

print(f'Test MSE for the Lasso model: {MSE_test3:.3f}')
print(f'Test R2 for the Lasso model: {R2_test3:.3f}')

Test MSE for the Lasso model: 0.098
Test R2 for the Lasso model: 0.275


In [15]:
# Store results
res_df2 = pd.DataFrame()

res_df2['Model'] = ['Basic Reg', 'Flexible Reg', 'Lasso']
res_df2['MSE_test'] = [MSE_test1, MSE_test2, MSE_test3]
res_df2['R2_test'] = [R2_test1, R2_test2, R2_test3]

res_df2.round(4)

,Model,MSE_test,R2_test
0,Basic Reg,0.1008,0.2561
1,Flexible Reg,0.0987,0.2717
2,Lasso,0.0983,0.2747


## (Optional) Extra flexible model



In [16]:
# Extra Flexible Model
model_extra = ('lwage ~ (female + age + race + children + marital '
              '+ education + RN_experience + RN_experience**2 + RN_experience**3 + RN_experience**4'
              '+ left_nursing + country_ed_US + english_only + military + certificates'
              '+ labor_union + work_setting + work_situation + level_care + care_specialty + state)**2')
tmp = smf.ols(model_extra, data=df) # just to extract df, not actually using this model
print(f'No_Features = {tmp.data.exog.shape[1]:.0f}')

# In-sample fit
insamplefit = tmp.fit()
rsquared_ex = insamplefit.rsquared
rsquared_adj_ex = insamplefit.rsquared_adj
mse_ex = np.mean(insamplefit.resid**2)
mse_adj_ex = insamplefit.mse_resid
print(f'(In-sample) Rsquared = {rsquared_ex :.3f}')
print(f'(In-sample) Rsquared_adjusted = {rsquared_adj_ex :.3f}')
print(f'(In-sample) MSE = {mse_ex :.3f}')
print(f'(In-sample) MSE_adjusted = {mse_adj_ex:.3f}')

# Train test Split
X_full = tmp.data.exog
y_full = tmp.data.endog
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=.2, random_state=42)

# Predict out of sample
reg_extra = sm.OLS(y_train, X_train).fit()
yhat_reg_extra = reg_extra.predict(X_test)

# Calculate out-of-sample MSE
MSE_test4 = np.mean((y_test - yhat_reg_extra)**2)
R2_test4 = 1. - MSE_test4 / np.var(y_test)

print(f'(Out-of-sample) MSE = {MSE_test4:.3f}')
print(f'(Out-of-sample) R2 = {R2_test4:.3f}')

No_Features = 3439
(In-sample) Rsquared = 0.740
(In-sample) Rsquared_adjusted = 0.413
(In-sample) MSE = 0.033
(In-sample) MSE_adjusted = 0.074
(Out-of-sample) MSE = 1.283
(Out-of-sample) R2 = -8.471


A simple OLS overfits when the dimensionality of covariates is high, while the out-of-sample performance suffers dramatically.

In [43]:
# Train model using Lasso
lasso = Pipeline([('scale', StandardScaler()),
                  ('lasso', LassoCV())])
lasso.fit(X_train[:, 1:], y_train)

# In-sample fit
yhat_train_lasso = lasso.predict(X_train[:, 1:])
R2_L = 1 - np.sum((yhat_train_lasso - y_train)**2) / np.sum((y_train - np.mean(y_train))**2)
pL = np.sum(lasso.named_steps['lasso'].coef_ != 0)
ntrain = len(X_train)
R2_adjL = 1 - (np.sum((yhat_train_lasso - y_train)**2) / (ntrain - pL - 1)) / (np.sum((y_train- np.mean(y_train))**2) / (ntrain - 1))
lasso_res = y_train - yhat_train_lasso
MSEL = np.mean(lasso_res**2)
MSE_adjL = (ntrain / (ntrain - pL - 1)) * MSEL
print(f'No_Nonzero_Features = {pL:.0f}')
print(f'(In-sample) Rsquared = {R2_L:.3f}')
print(f'(In-sample) Rsquared_adjusted = {R2_adjL:.3f}')
print(f'(In-sample) MSE = {MSEL:.3f}')
print(f'(In-sample) MSE_adjusted = {MSE_adjL:.3f}')

# Out-of-sample fit
yhat_test_lasso = lasso.predict(X_test[:, 1:])
MSE_test5 = np.mean((y_test - yhat_test_lasso)**2)
R2_test5 = 1. - MSE_test5 / np.var(y_test)

print(f'(Out-of-sample) MSE = {MSE_test5:.3f}')
print(f'(Out-of-sample) R2 = {R2_test5:.3f}')

No_Nonzero_Features = 271
(In-sample) Rsquared = 0.370
(In-sample) Rsquared_adjusted = 0.329
(In-sample) MSE = 0.078
(In-sample) MSE_adjusted = 0.083
(Out-of-sample) MSE = 0.106
(Out-of-sample) R2 = 0.219


Overfitting is mitigated with the penalized regression model.

## Ridge and Elastic Net

In addition to Lasso, we can also explore Ridge and Elastic Net regression models to see how they perform in terms of prediction accuracy. 

Ridge regression adds a penalty to the loss function that shrinks the coefficients toward zero, but without allowing any of them to be exactly zero, thus reducing their variance while retaining all predictors. 

Elastic net combines the penalties of both ridge and lasso regression, incorporating a mix of the ridge penalty (which shrinks coefficients) and the lasso penalty (which can set some coefficients exactly to zero). This hybrid approach is useful when dealing with highly correlated predictors, as it can select a subset of predictors while also shrinking coefficients to prevent overfitting.

In [16]:
# Ridge regression
ridge = Pipeline(steps=[('scaler', StandardScaler()),
                        ('ridge', RidgeCV())])
ridge.fit(X_train[:, 1:], y_train)
yhat_test_ridge = ridge.predict(X_test[:, 1:])

# Calculate out-of-sample MSE
MSE_test_ridge = np.mean((y_test - yhat_test_ridge)**2)
R2_test_ridge = 1. - MSE_test_ridge / np.var(y_test)

print(f'Test MSE for Ridge model: {MSE_test_ridge:.3f}')
print(f'Test R2 for Ridge model: {R2_test_ridge:.3f}')

Test MSE for Ridge model: 0.099
Test R2 for Ridge model: 0.271


In [17]:
# Elastic Net
elastic = Pipeline(steps=[('scaler', StandardScaler()),
                          ('elastic', ElasticNetCV())])
elastic.fit(X_train[:, 1:], y_train)
yhat_test_elastic = elastic.predict(X_test[:, 1:])

# Calculate out-of-sample MSE
MSE_test_elastic = np.mean((y_test - yhat_test_elastic)**2)
R2_test_elastic = 1. - MSE_test_elastic / np.var(y_test)

print(f'Test MSE for Elastic Net model: {MSE_test_elastic:.3f}')
print(f'Test R2 for Elastic Net model: {R2_test_elastic:.3f}')

Test MSE for Elastic Net model: 0.098
Test R2 for Elastic Net model: 0.274


In [18]:
# Store results
res_df2.loc[3] = ['Ridge', MSE_test_ridge, R2_test_ridge]
res_df2.loc[4] = ['Elastic Net', MSE_test_elastic, R2_test_elastic]

res_df2.round(4)

,Model,MSE_test,R2_test
0,Basic Reg,0.1008,0.2561
1,Flexible Reg,0.0987,0.2717
2,Lasso,0.0983,0.2747
3,Ridge,0.0987,0.2710
4,Elastic Net,0.0983,0.2741


## Lasso with theoretical-optimal $\lambda$ and Post-Lasso

This is a based on a Python implementation made by [Max Huppertz](https://maxhuppertz.github.io/code/). His library is located in this [repository](https://github.com/maxhuppertz/hdmpy).

Run the following code to install the library:

```python
!git clone https://github.com/maxhuppertz/hdmpy.git
!pip install multiprocess
```

In [19]:
import hdmpy

# Wrap the package so that it has the familiar sklearn API
class RLasso(BaseEstimator, RegressorMixin):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    @property
    def coef_(self):
        return np.array(self.rlasso_.est['beta']).flatten()

    @property
    def intercept_(self):
        return np.array(self.rlasso_.est['intercept'])

    def predict(self, X):
        return X @ self.coef_ + self.intercept_

In [20]:
# Robust Lasso
rlasso = Pipeline(steps=[('scaler', StandardScaler()),
                         ('rlasso', RLasso(post=False))])
rlasso.fit(X_train[:, 1:], y_train)
yhat_test_rlasso = rlasso.predict(X_test[:, 1:])
MSE_test_rlasso = np.mean((y_test - yhat_test_rlasso)**2)
R2_test_rlasso = 1. - MSE_test_rlasso / np.var(y_test)

# Robust Post Lasso
rlasso_post = Pipeline(steps=[('scaler', StandardScaler()),
                               ('rlasso', RLasso(post=True))])
rlasso_post.fit(X_train[:, 1:], y_train)
yhat_test_rlasso_post = rlasso_post.predict(X_test[:, 1:])
MSE_test_rlasso_post = np.mean((y_test - yhat_test_rlasso_post)**2)
R2_test_rlasso_post = 1. - MSE_test_rlasso_post / np.var(y_test)

# Store results
res_df2.loc[5] = ['Robust Lasso', MSE_test_rlasso, R2_test_rlasso]
res_df2.loc[6] = ['Robust Post Lasso', MSE_test_rlasso_post, R2_test_rlasso_post]

res_df2.round(4)

,Model,MSE_test,R2_test
0,Basic Reg,0.1008,0.2561
1,Flexible Reg,0.0987,0.2717
2,Lasso,0.0983,0.2747
3,Ridge,0.0987,0.2710
4,Elastic Net,0.0983,0.2741
5,Robust Lasso,0.1032,0.2380
6,Robust Post Lasso,0.1053,0.2225
